In [488]:
import pandas as pd
import numpy as np
import json

In [489]:
# The principal manual cleaning was done on the video-games-developers.csv and worldcities.csv to match the other datasets

# Correcting the labels of games.csv
corrected_labels ="AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLCcount,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies\n"
modified = False
with open("Data/Raw_Data/games.csv", 'r', encoding='utf-8') as f_in:
    with open("Data/Modified_Raw_Data/games.csv", 'w', encoding='utf-8') as f_out:
        for l in f_in:
            if not modified:
                f_out.write(corrected_labels)
                modified = True
            else:
                f_out.write(l)

In [490]:
with open("Data/Modified_Raw_Data/modif_video-games-developers.json", 'r', encoding='utf-8') as f_in:
    modifications = json.load(f_in)

dico_updates = { item["index"]: item["line"] for item in modifications }

with open("Data/Raw_Data/video-games-developers.csv", 'r', encoding='utf-8') as f_in, \
        open("Data/Modified_Raw_Data/video-games-developers.csv", 'w', encoding='utf-8') as f_out:
    
    for i, ligne_originale in enumerate(f_in):
        if i in dico_updates:
            f_out.write(dico_updates[i])
        else:
            f_out.write(ligne_originale)


In [491]:
df_games = pd.read_csv("Data\Modified_Raw_Data\games.csv", sep=",")
df_studios = pd.read_csv("Data/Modified_Raw_Data/video-games-developers.csv", sep=",")
df_countries = pd.read_csv("Data/Raw_Data/countries of the world.csv", sep=",")
df_cities = pd.read_csv("Data/Raw_Data/worldcities.csv", sep=",")


<>:1: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
<>:1: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
C:\Users\kelia\AppData\Local\Temp\ipykernel_19448\1107613088.py:1: SyntaxWarning: "\M" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\M"? A raw string is also an option.
  df_games = pd.read_csv("Data\Modified_Raw_Data\games.csv", sep=",")


In [492]:
# For games, we keep the following columns only
df_games = df_games[['AppID', 'Name', 'Release date', 'Estimated owners',
'Required age', 'Price', 'DLCcount',
'Supported languages','Windows','Mac', 'Linux', 'Metacritic score', 'User score',
'Positive', 'Negative', 'Achievements','Average playtime forever',
'Developers','Categories', 'Genres']]

# Cleaning Supported languages column
df_games['Supported languages'] = df_games['Supported languages'].str.replace("'", '').str.replace("[", '').str.replace("]", '')

# Checking for the Primary Key
print(df_games.duplicated(subset=['AppID'], keep=False).sum())

df_games.head(2)


0


,AppID,Name,Release date,Estimated owners,Required age,Price,DLCcount,Supported languages,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Achievements,Average playtime forever,Developers,Categories,Genres
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,19.99,0,English,True,False,False,0,0,6,11,30,0,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports"
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0.99,0,"English, French, Italian, German, Spanish - Sp...",True,True,False,0,0,53,5,12,0,Rusty Moyher,"Single-player,Steam Achievements,Full controll...","Action,Indie"


In [493]:
# Checking for the Primary Key
print(df_countries.duplicated(subset=['Country'], keep=False).sum())

df_countries.head(2)


0


,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"


In [494]:
# Checking for the Primary Key
print(df_cities.duplicated(subset=['id'], keep=False).sum())

df_cities.head(2)

0


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.687,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
1,Jakarta,Jakarta,-6.175,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077


In [495]:
# Checking for the Primary Key
print(df_studios.duplicated(subset=['Developer'], keep=False).sum())

df_studios.sort_values('Developer', inplace=True)
id = range(0, len(df_studios))
df_studios['DeveloperID'] = id


df_studios.head(2)

0


,Developer,City,Administrative division,Country,Est.,"Notable games, series or franchises",Notes,DeveloperID
0,0verflow,Tokyo,NaN,Japan,1997,School DaysSummer DaysCross Days,Visual Novel brand (both developer and publisher),0
3,1-Up Studio,Tokyo,NaN,Japan,2000,Mother 3,Subsidiary of Nintendo. Formed by former emplo...,1


In [496]:
# Checking Foreign Key between game and studios & indies

# We drop the rows with missing values for Developers
df_games.dropna(subset=['Developers'], inplace=True)

print(df_games['Developers'].unique().shape)

# We keep only the rows with developers that are in studios or indies. Definition of the foreign key.
df_games = df_games[df_games.Developers.isin(df_studios.Developer)]



(64655,)


In [497]:
list_cat = []
list_genres = []
list_languages = []

for cat,genre,lang in zip(df_games['Categories'], df_games['Genres'], df_games['Supported languages']):
    if cat is not np.nan:
        for c in cat.split(','):
            if c not in list_cat:
                list_cat.append(c)
    if genre is not np.nan:
        for g in genre.split(','):
            if g not in list_genres:
                list_genres.append(g)
    if lang is not np.nan:
        for l in lang.split(','):
            if l not in list_languages:
                list_languages.append(l)


list_cat.sort()
list_genres.sort()
list_languages.sort()

df_cat = pd.DataFrame({'Categories': list_cat, 'CategoryID': range(len(list_cat))})
df_genres = pd.DataFrame({'Genres': list_genres, 'GenreID': range(len(list_genres))})
df_languages = pd.DataFrame({'Languages': list_languages, 'LanguageID': range(len(list_languages))})

df_cat.to_csv('Data/Clean_Data/categories.csv', index=False)
df_genres.to_csv('Data/Clean_Data/genres.csv', index=False)
df_languages.to_csv('Data/Clean_Data/langues.csv', index=False)

categories_to_games = []
genres_to_games = []
languages_to_games = []

for cat,genre,appID in zip(df_games['Categories'], df_games['Genres'], df_games['AppID']):
    if cat is not np.nan:
        for c in cat.split(','):
            catID = list_cat.index(c)
            categories_to_games.append([appID, catID])
    if genre is not np.nan:
        for g in genre.split(','):
            gID = list_genres.index(g)
            genres_to_games.append([appID, gID])
    if lang is not np.nan:
        for l in lang.split(','):
            lID = list_languages.index(l)
            languages_to_games.append([appID, lID])

df_categories_to_games = pd.DataFrame(categories_to_games, columns=['AppID', 'CategoryID'])
df_genres_to_games = pd.DataFrame(genres_to_games, columns=['AppID', 'Genre'])
df_languages_to_games = pd.DataFrame(languages_to_games, columns=['AppID', 'LanguageID'])

df_categories_to_games.to_csv('Data/Clean_Data/games_categories.csv', index=False)
df_genres_to_games.to_csv('Data/Clean_Data/games_genres.csv', index=False)
df_languages_to_games.to_csv('Data/Clean_Data/games_languages.csv', index=False)

In [ ]:
# Creating mutliple tables from games dataset
# df_games.to_csv('Data/Clean_Data/games.csv', index=False)

# Release infos:

# Developers infos:
list_dev_Id = []
for dev in df_games['Developers']:
    list_dev_Id.append(df_studios[df_studios['Developer'] == dev]['DeveloperID'].values[0])

df_games['DevelopersID'] = list_dev_Id

df_games[['AppID', 'Name','DevelopersID', 'Release date','Required age', 
'Price', 'DLCcount','Supported languages','Windows','Mac', 'Linux',
'Achievements','Estimated owners','Metacritic score', 'User score', 
'Positive', 'Negative', 'Average playtime forever']].to_csv('Data/Clean_Data/games.csv', index=False)

In [499]:
print(df_cities.duplicated(subset=['city_ascii','country'], keep=False).sum())

3819


In [500]:
l = 0
city_id_list = []

for city,country in zip(df_studios['City'], df_studios['Country']):
    if df_cities[(df_cities['city_ascii'] == city) | (df_cities['admin_name'] == city) | (df_cities['city'] == city)].shape[0] == 0:
        if city is not np.nan:
            print(f'{l} : No match')
            print(city,'-',country)
            l += 1
        city_id_list.append("")
    else:
        city_id = df_cities[(df_cities['city_ascii'] == city) | (df_cities['admin_name'] == city) | (df_cities['city'] == city) & (df_cities['country'] == country)].sort_values('population', ascending=False)['id'].iloc[0]
        city_id_list.append(city_id)
    
df_studios['CityID'] = city_id_list


In [501]:
# Creating mutliple tables from games dataset

# Notes infos:
df_studios[['DeveloperID','Developer','Notable games, series or franchises','Notes','CityID','Administrative division','Country','Est.']].to_csv('Data/Clean_Data/studios.csv', index=False)



In [502]:
df_cities[["id","city","city_ascii","lat","lng","country","iso2","iso3","admin_name","capital","population"]].to_csv('Data/Clean_Data/cities.csv', index=False)

In [503]:
drop_countries = [] 


df_countries['Country'] = df_countries['Country'].str.strip()
for country in df_cities['country'].unique():
    if country not in df_countries['Country'].unique():
        drop_countries.append(country)

    
df_cities = df_cities[~df_cities['country'].isin(drop_countries)]

In [504]:
l = 0
for country in df_studios['Country'].unique():
    if country not in df_countries['Country'].unique():
        print(l,country)
        l += 1


In [507]:
df_countries['Region'] = df_countries['Region'].str.strip()

In [508]:
df_countries.to_csv('Data/Clean_Data/countries.csv', index=False)